# CatBoostClasiffier

En este notebook, se modela el entrenamiento y testing de un modelo `CatBoostClasiffier` sobre el dataset obtenido del preprocesado, que incluye la obtención de dummies. Gracias al buen desempeño de CatBoost con variables categóricas, numéricas (binarias) y booleanas, se analizará el desempeño de una versión simple del modelo. Además, se analizará la importancia de las categorías utilizadas por si resultara relevante para el entrenamiento de otros modelos y la posibilidad de añadir variables categóricas que representen el efecto agrupado de varias variables. 

In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import datetime
import pandas as pd
from catboost import Pool

In [2]:
# Cargar los datos
df = pd.read_csv("../../data/processed/data_simp_preprocess_v2.csv")
df.head(3)

,id,label,statement,subject,speaker,party_affiliation,party_affiliation_uni,party_affiliation_category_map,economy,health-care,...,state_info_texas,state_info_united_states,state_info_utah,state_info_vermont,state_info_virginia,state_info_washington,state_info_washington_dc,state_info_west_virginia,state_info_wisconsin,state_info_wyoming
0,81f884c64a7,1.0,china is in the south china sea and (building)...,"china,foreign-policy,military",donald-trump,republican,republican,political-affiliation,0,0,...,False,False,False,False,False,False,False,False,False,False
1,30c2723a188,0.0,with the resources it takes to execute just ov...,health-care,chris-dodd,democrat,democrat,political-affiliation,0,1,...,False,False,False,False,False,False,False,False,False,False
2,6936b216e5d,0.0,the (wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,democrat,democrat,political-affiliation,0,0,...,False,False,False,False,False,False,True,False,False,False


Extraemos las etiquetas, descartamos las variables que carecen de valor predictivo (`id`) y aquellas que han sido procesadas en dummies

In [3]:
drop_cols = ["id","statement","subject","label","speaker","party_affiliation","party_affiliation_category_map"]

In [4]:
df_label = df["label"].astype(int)
df = df.drop(columns=drop_cols)

## Modelo Catboost Simple

Inicialmente, entrenamos el modelo con todas las columnas del dataset. 

In [5]:
# Definir la columna objetivo
X = df.copy()
y = df_label

# Detectar columnas categóricas
cat_features = X.select_dtypes(include=['object']).columns.tolist()
cat_features += ["speaker_job_grouped"]

Estratificamos el dataset asegurando que la proporción en ambas matrices resultado sea la misma que en la del dataset original

In [6]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Dimensiones del conjunto de entrenamiento y test:")
print(X_train.shape)
print(X_test.shape)
print("Proporciones de la clase objetivo.\nOriginal:")
print(df_label.value_counts(normalize=True))
print("Conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))
print("Conjunto de test:")
print(y_test.value_counts(normalize=True))

Dimensiones del conjunto de entrenamiento y test:
(7160, 189)
(1790, 189)
Proporciones de la clase objetivo.
Original:
label
1    0.647486
0    0.352514
Name: proportion, dtype: float64
Conjunto de entrenamiento:
label
1    0.647486
0    0.352514
Name: proportion, dtype: float64
Conjunto de test:
label
1    0.647486
0    0.352514
Name: proportion, dtype: float64


El modelo entrenado inicialmente presenta un enfoque estándar. Dado que hemos observado que el dataset está desbalanceado a favor de la clase 1, se debe aplicar balanceo de la clase objetivo. Para ello, se han probado dos técnicas:
- Balanceo nativo en CatBoost mediante el parámetro 'auto_class_weights'
- SMOTE

Finalmente, se ilustra la primera opción, que obtuvo los mejores resultados. 

In [7]:
# Inicializar y entrenar el modelo
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.03,
    depth=8,
    cat_features=cat_features,
    auto_class_weights='Balanced',
    verbose=100,
    random_state=42
)
model.fit(X_train, y_train)

0:	learn: 0.6922372	total: 71.9ms	remaining: 35.9s
100:	learn: 0.6540164	total: 1.63s	remaining: 6.45s
200:	learn: 0.6352498	total: 2.92s	remaining: 4.35s
300:	learn: 0.6194425	total: 4.26s	remaining: 2.81s
400:	learn: 0.5960014	total: 5.59s	remaining: 1.38s
499:	learn: 0.5754481	total: 6.91s	remaining: 0us


In [8]:
# Evaluar el modelo
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6011173184357542
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.62      0.52       631
           1       0.74      0.59      0.66      1159

    accuracy                           0.60      1790
   macro avg       0.60      0.61      0.59      1790
weighted avg       0.64      0.60      0.61      1790



- **Modelo CatBoost Simple** con `auto_class_weights`: `macro-avg=0,59`, `precision[0]=45` y `precision[1]=0,73`. *Sol*: Reducir número de variables para ver si el modelo mantiene la generalización (equivalente a eliminar cardinalidad y ruido)
- **Modelo CatBoost Simple** sin `party_affiliation`: `macro-avg`baja un 1%, bajando en la misma medida la precisión de la clase minoritaria. 
- **Modelo CatBoost Simple** sin `party_affiliation` ni `party_affiliation_uni`: `macro-avg`baja un 1%, bajando en la misma medida la precisión de la clase mayoritaria.

En todos los casos, el modelo presenta un peor desempeño en la detección de la clase minoritaria.

## Ajuste de Hiperparámetros con RandomizedSearch

Buscamos los mejores hiperparámetros para el modelo simplificado inicial

In [9]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
import numpy as np


# Modelo base
base_model = CatBoostClassifier(
    auto_class_weights='Balanced',
    verbose=100,
    random_state=42,
    early_stopping_rounds = 30
)

# Hiperparámetros a explorar
param_dist = {
    'depth': [4, 6],
    'learning_rate': np.linspace(0.01, 0.05, 5),
    'iterations': [500, 800, 1000],
    'l2_leaf_reg': [5, 10, 20],
    'min_data_in_leaf': [10, 20, 30],
    'bagging_temperature': [0.5, 1, 1.5],
    'random_strength': [0.5, 1, 1.5]
}


stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Búsqueda aleatoria
search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='f1_macro',
    cv=stratified_cv,
    verbose=3,
    n_jobs=-1
)


In [10]:
# Iniciamos la busqueda
search.fit(X_train, y_train, cat_features=cat_features)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0:	learn: 0.6923926	total: 8.22ms	remaining: 4.1s
100:	learn: 0.6670019	total: 725ms	remaining: 2.87s
200:	learn: 0.6577691	total: 1.44s	remaining: 2.13s
300:	learn: 0.6498187	total: 2.14s	remaining: 1.42s
400:	learn: 0.6376227	total: 2.87s	remaining: 708ms
499:	learn: 0.6267062	total: 3.58s	remaining: 0us


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=<catboost.core.CatBoostClassifier object at 0x12d005d80>,
                   n_iter=20, n_jobs=-1,
                   param_distributions={'bagging_temperature': [0.5, 1, 1.5],
                                        'depth': [4, 6],
                                        'iterations': [500, 800, 1000],
                                        'l2_leaf_reg': [5, 10, 20],
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05]),
                                        'min_data_in_leaf': [10, 20, 30],
                                        'random_strength': [0.5, 1, 1.5]},
                   scoring='f1_macro', verbose=3)

In [11]:
# Evaluación final en test
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
print("Best Params:", search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Best Params: {'random_strength': 1.5, 'min_data_in_leaf': 30, 'learning_rate': 0.03, 'l2_leaf_reg': 10, 'iterations': 500, 'depth': 6, 'bagging_temperature': 0.5}
Accuracy: 0.5871508379888268
Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.60      0.51       631
           1       0.73      0.58      0.65      1159

    accuracy                           0.59      1790
   macro avg       0.58      0.59      0.58      1790
weighted avg       0.62      0.59      0.60      1790



#### Subida a Kaggle 

In [12]:
cat_features

['party_affiliation_uni', 'speaker_job_grouped', 'speaker_job_grouped']

In [13]:
# Cargar los datos de test
df_test = pd.read_csv("../../data/processed/test_simp_preprocess_v1.csv")

# Guardar id
test_ids = df_test["id"]

Resulta de vital importancia que el conjunto de test contenga el mismo procesado que el de entrenamiento, por lo que se eliminan las mismas columnas que en el dataset inicial

In [14]:
# Eliminar las mismas columnas que en df
drop_cols_test =  drop_cols.copy()
print(drop_cols_test)

drop_cols_test.remove("label")
drop_cols_test.remove("party_affiliation")
print(drop_cols_test)

df_test = df_test.drop(columns=drop_cols_test)
df_test.head(2)

['id', 'statement', 'subject', 'label', 'speaker', 'party_affiliation', 'party_affiliation_category_map']
['id', 'statement', 'subject', 'speaker', 'party_affiliation_category_map']


,party_affiliation_uni,economy,health-care,taxes,federal-budget,education,jobs,state-budget,candidates-biography,elections,...,party_affiliation_columnist,party_affiliation_democrat,party_affiliation_independent,party_affiliation_journalist,party_affiliation_libertarian,party_affiliation_newsmaker,party_affiliation_none,party_affiliation_organization,party_affiliation_other,party_affiliation_republican
0,democrat,0,0,0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
1,republican,0,0,0,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,True


Debido a que el procesado mediante dummies codifica únicamente los valores presentes en el dataset, las columnas faltantes en el conjunto de test que estén presentes en el dataset original se deben incluir.

In [15]:
# Ajuste de columnas (por dummies)
missing_cols = set(df.columns) - set(df_test.columns)
for col in missing_cols:
    df_test[col] = 0  # Rellenar columnas faltantes con 0

# Avisar si faltan columnas por transparencia
if missing_cols:
    print(f"Columnas faltantes añadidas en test: {len(missing_cols)}")

Columnas faltantes añadidas en test: 54


In [16]:
cat_features_test = df_test.select_dtypes(include=['object']).columns.tolist()
cat_features_test
#df_test = df_test.drop(columns=['speaker_job_grouped'])

['party_affiliation_uni', 'speaker_job_grouped']

In [17]:
# Crear el Pool
test_pool = Pool(df_test, cat_features=cat_features)
y_pred_test = best_model.predict(test_pool)

# Guardar predicciones
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output = pd.DataFrame({
    "id": test_ids,
    "label": y_pred_test.astype(int)
})
filename = f"../3_summision/CatBoost_Simple{current_date}.csv"
output.to_csv(filename, index=False)
print(f"Predicciones guardadas en {filename}")

Predicciones guardadas en ../3_summision/CatBoost_Simple2025-05-20_20-35-50.csv


#### Resultados:
1. Dataset con todas las categorías procesado con CatBoost Simple + Balanceo de clases + Búsqueda de Hiperparámetros: **0.55996**
2. Con Randomized más robusto y StratifiedFolds sube a **0.56244**. Aumenta la detección de la clase minoritaria y la diferencia entre el resultado de Kaggle y el notebook baja considerablemente, indicando que el modelo está controlando bien el overfitting. 

## Ajuste de umbral

In [18]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Obtener probabilidades de clase 1
y_probs = best_model.predict_proba(X_test)[:, 1]

# Probar distintos umbrales
thresholds = np.arange(0.1, 0.91, 0.01)
f1_scores = []

for t in thresholds:
    y_pred_thresh = (y_probs >= t).astype(int)
    score = f1_score(y_pred, y_pred_thresh, average='macro')
    f1_scores.append(score)

# Mejor umbral
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

# Evaluar con el mejor umbral
y_pred_opt = (y_probs >= best_threshold).astype(int)

print(f"Mejor umbral: {best_threshold:.2f} con F1 macro: {best_f1:.4f}")
print("Best Params:", search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_opt))
print("Classification Report:\n", classification_report(y_test, y_pred_opt))


Mejor umbral: 0.50 con F1 macro: 1.0000
Best Params: {'random_strength': 1.5, 'min_data_in_leaf': 30, 'learning_rate': 0.03, 'l2_leaf_reg': 10, 'iterations': 500, 'depth': 6, 'bagging_temperature': 0.5}
Accuracy: 0.5871508379888268
Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.60      0.51       631
           1       0.73      0.58      0.65      1159

    accuracy                           0.59      1790
   macro avg       0.58      0.59      0.58      1790
weighted avg       0.62      0.59      0.60      1790



El ajuste de umbral no producirá ninguna mejora, ya que el mejor umbral es el marcado por defecto

#### Subida a Kaggle

In [19]:
# Cargar los datos de test
df_test = pd.read_csv("../../data/processed/test_simp_preprocess_v1.csv")

# Guardar id
test_ids = df_test["id"]

In [20]:
# Crear el Pool
y_probs_test = best_model.predict_proba(test_pool)[:, 1]
# Evaluar con el mejor umbral
y_pred_test = (y_probs_test >= best_threshold).astype(int)

# Guardar predicciones
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output = pd.DataFrame({
    "id": test_ids,
    "label": y_pred_test.astype(int)
})
filename = f"../3_summision/CatBoost_Simple_Threshold_{current_date}.csv"
output.to_csv(filename, index=False)
print(f"Predicciones guardadas en {filename}")

Predicciones guardadas en ../3_summision/CatBoost_Simple_Threshold_2025-05-20_20-35-50.csv


## Importancia de variables

In [21]:
# Obtener importancia de características
feature_importances = best_model.get_feature_importance(prettified=True)
feature_importances.columns = ['feature', 'importance']
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

,feature,importance
0,party_affiliation_uni,20.419284
1,speaker_job_grouped,7.069121
2,health-care,5.579433
3,speaker_job_non-define,3.052355
4,jobs,2.802975
...,...,...
180,state_info_qatar,0.000000
179,state_info_oklahoma,0.000000
177,state_info_massachusetts,0.000000
176,state_info_hawaii,0.000000


Vemos que `party_affiliation_uni` contiene información muy relevante para el modelo, quizás tomando demasiado protagonismo. Se considera, para iteraciones futuras, quitar esta variable y ver si 'party_affiliation_category_map` toma importancia suficiente, sin perder el efecto de la afiliación política en el modelo. 

In [22]:
# Filtramos las más relevantes
top_n = 100
top_features = feature_importances['feature'].head(top_n).tolist()

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]
cat_features_top = X_train_top.select_dtypes(include=['object']).columns.tolist()
cat_features_top

['party_affiliation_uni', 'speaker_job_grouped']

In [23]:
# Buscamos el modelo con mejores hiperparámetros
search_top = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='f1_macro',
    cv=3,
    verbose=3,
    n_jobs=-1
)
search_top.fit(X_train_top, y_train, cat_features=cat_features_top)
best_model_top = search_top.best_estimator_

params_top = best_model_top.get_params()

# Evaluar el modelo
y_pred_top = best_model_top.predict(X_test_top)
print("Accuracy:", accuracy_score(y_test, y_pred_top))
print("Classification Report:\n", classification_report(y_test, y_pred_top))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
0:	learn: 0.6915914	total: 7.53ms	remaining: 6.01s
100:	learn: 0.6488012	total: 734ms	remaining: 5.08s
200:	learn: 0.6281607	total: 1.45s	remaining: 4.31s
300:	learn: 0.6079386	total: 2.18s	remaining: 3.62s
400:	learn: 0.5909764	total: 2.9s	remaining: 2.88s
500:	learn: 0.5777580	total: 3.61s	remaining: 2.15s
600:	learn: 0.5649198	total: 4.37s	remaining: 1.45s
700:	learn: 0.5551164	total: 5.07s	remaining: 715ms
799:	learn: 0.5457126	total: 5.79s	remaining: 0us
Accuracy: 0.5748603351955307
Classification Report:
               precision    recall  f1-score   support

           0       0.42      0.58      0.49       631
           1       0.71      0.57      0.64      1159

    accuracy                           0.57      1790
   macro avg       0.57      0.58      0.56      1790
weighted avg       0.61      0.57      0.58      1790



El resultado se mantiene, pero aumenta la detección de la clase 0. 
#### Subida a Kaggle

In [24]:
# Cargar los datos de test
df_test = pd.read_csv("../../data/processed/test_simp_preprocess_v1.csv")

# Guardar id
test_ids = df_test["id"]


In [25]:
# Eliminar las mismas columnas que en df
drop_cols_test =  drop_cols.copy()
print(drop_cols_test)

drop_cols_test.remove("label")
drop_cols_test.remove("party_affiliation")
print(drop_cols_test)

df_test = df_test.drop(columns=drop_cols_test)
df_test.head(2)

['id', 'statement', 'subject', 'label', 'speaker', 'party_affiliation', 'party_affiliation_category_map']
['id', 'statement', 'subject', 'speaker', 'party_affiliation_category_map']


,party_affiliation_uni,economy,health-care,taxes,federal-budget,education,jobs,state-budget,candidates-biography,elections,...,party_affiliation_columnist,party_affiliation_democrat,party_affiliation_independent,party_affiliation_journalist,party_affiliation_libertarian,party_affiliation_newsmaker,party_affiliation_none,party_affiliation_organization,party_affiliation_other,party_affiliation_republican
0,democrat,0,0,0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
1,republican,0,0,0,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,True


In [26]:
# Ajuste de columnas (por dummies)
missing_cols = set(df.columns) - set(df_test.columns)
for col in missing_cols:
    df_test[col] = 0  # Rellenar columnas faltantes con 0

# Avisar si faltan columnas por transparencia
if missing_cols:
    print(f"Columnas faltantes añadidas en test: {len(missing_cols)}")

Columnas faltantes añadidas en test: 54


In [27]:
cat_features_test = df_test.select_dtypes(include=['object']).columns.tolist()
cat_features_test
#df_test = df_test.drop(columns=['speaker_job_grouped'])

['party_affiliation_uni', 'speaker_job_grouped']

In [28]:
# Crear el Pool
test_pool = Pool(df_test, cat_features=cat_features)
y_pred_top_test = best_model_top.predict(test_pool)

# Guardar predicciones
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output = pd.DataFrame({
    "id": test_ids,
    "label": y_pred_top_test.astype(int)
})
filename = f"../3_summision/CatBoost_Simple_Importancia{current_date}.csv"
output.to_csv(filename, index=False)
print(f"Predicciones guardadas en {filename}")

Predicciones guardadas en ../3_summision/CatBoost_Simple_Importancia2025-05-20_20-38-34.csv


#### Resultados:
1. Dataset con TOP100 categorías procesado con CatBoost Simple + Balanceo de clases + Búsqueda de Hiperparámetros: **0.56539**. Sube aproximadamente un 1% más que con todas las categorías: no solo generaliza mejor sino que además categorías por debajo del 100 introducen ruido.


### Ajuste de umbral

In [29]:
# Obtener probabilidades de clase 1
y_probs = best_model_top.predict_proba(X_test_top)[:, 1]

# Probar distintos umbrales
thresholds = np.arange(0.1, 0.91, 0.01)
f1_scores = []

for t in thresholds:
    y_pred_thresh = (y_probs >= t).astype(int)
    score = f1_score(y_test, y_pred_thresh, average='macro')
    f1_scores.append(score)

# Mejor umbral
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

# Evaluar con el mejor umbral
y_pred_opt = (y_probs >= best_threshold).astype(int)

print(f"Mejor umbral: {best_threshold:.2f} con F1 macro: {best_f1:.4f}")
print("Best Params:", search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_opt))
print("Classification Report:\n", classification_report(y_test, y_pred_opt))



Mejor umbral: 0.42 con F1 macro: 0.5798
Best Params: {'random_strength': 1.5, 'min_data_in_leaf': 30, 'learning_rate': 0.03, 'l2_leaf_reg': 10, 'iterations': 500, 'depth': 6, 'bagging_temperature': 0.5}
Accuracy: 0.6374301675977654
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.38      0.42       631
           1       0.70      0.78      0.74      1159

    accuracy                           0.64      1790
   macro avg       0.59      0.58      0.58      1790
weighted avg       0.62      0.64      0.63      1790



#### Subir a Kaggle

In [30]:
# Cargar los datos de test
df_test = pd.read_csv("../../data/processed/test_simp_preprocess_v1.csv")

# Guardar id
test_ids = df_test["id"]

In [31]:
# Eliminar las mismas columnas que en df
drop_cols_test =  drop_cols.copy()
print(drop_cols_test)

drop_cols_test.remove("label")
drop_cols_test.remove("party_affiliation")
print(drop_cols_test)

df_test = df_test.drop(columns=drop_cols_test)
df_test.head(2)

['id', 'statement', 'subject', 'label', 'speaker', 'party_affiliation', 'party_affiliation_category_map']
['id', 'statement', 'subject', 'speaker', 'party_affiliation_category_map']


,party_affiliation_uni,economy,health-care,taxes,federal-budget,education,jobs,state-budget,candidates-biography,elections,...,party_affiliation_columnist,party_affiliation_democrat,party_affiliation_independent,party_affiliation_journalist,party_affiliation_libertarian,party_affiliation_newsmaker,party_affiliation_none,party_affiliation_organization,party_affiliation_other,party_affiliation_republican
0,democrat,0,0,0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
1,republican,0,0,0,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,True


In [32]:
# Ajuste de columnas (por dummies)
missing_cols = set(df.columns) - set(df_test.columns)
for col in missing_cols:
    df_test[col] = 0  # Rellenar columnas faltantes con 0

# Avisar si faltan columnas por transparencia
if missing_cols:
    print(f"Columnas faltantes añadidas en test: {len(missing_cols)}")

Columnas faltantes añadidas en test: 54


In [33]:
cat_features_test = df_test.select_dtypes(include=['object']).columns.tolist()
cat_features_test
#df_test = df_test.drop(columns=['speaker_job_grouped'])

['party_affiliation_uni', 'speaker_job_grouped']

In [34]:
# Crear el Pool
test_pool = Pool(df_test, cat_features=cat_features)
y_probs = best_model_top.predict_proba(test_pool)[:, 1]

# Evaluar con el mejor umbral
y_pred_test = (y_probs >= best_threshold).astype(int)

# Guardar predicciones
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output = pd.DataFrame({
    "id": test_ids,
    "label": y_pred_test.astype(int)
})
filename = f"../3_summision/CatBoost_Simple_top_Threshold{current_date}.csv"
output.to_csv(filename, index=False)
print(f"Predicciones guardadas en {filename}")

Predicciones guardadas en ../3_summision/CatBoost_Simple_top_Threshold2025-05-20_20-38-35.csv


## Conclusiones: 


El dataset utilizado para entrenar el modelo no contiene las variables: 
- `statement`, cuya importancia, a pesar de tratar de paliarla con ingeniería de características (agrupación, mayormente) no se ve reducida, por lo que queda 15 puntos porcentuales por encima de la siguiente variable de importancia. Ante la imposibilidad de reducir la inclinación del modelo a no decidir en base a esta variable, se elimina.
- `party_affiliation` y su versión `_category_map`, que contiene un mapeo manual, ya que el efecto del partido político se encuentra reflejado en la variable `party_affiliation_uni`. Esta resulta decisiva para el modelo, y presenta cardinalidad adecuada para que CatBoost pueda procesarla como variable categórica.
- `speaker`, de la cual, durante la fase de preprocesado, se extrajeron características relevantes representadas en el dataset como 'dummies'. Estas finalmente también fueron eliminadas. En su lugar, se ha optado por un mapeo agrupado en `speaker_job`, que parece adecuada para la predicción por su importancia en el modelo.

Para el modelado, se utilizó: 
- Estratificación del dataset, en conjunto de entrenamiento y test para poder evaluar y maximizar la métrica f1-score.
- Balanceo automático nativo en CatBoost mediante la variable 'auto_class_weights'
- `RandomizedSearchcv` Y `StratifiedKFolds` para búsqueda de hiperparámetros en 5 conjuntos de datos.
- Importancia de variables para la selección de variables relevantes para el aprendizaje del modelo, buscando reducir el ruido y la cardinalidad del dataset sin perjudicar los resultados.
- Ajuste de umbral para evitar la predilección del modelo por la clase mayoritaria.

Con todo ello, los resultados obtenidos fueron: 
- Sensiblemente mejores al filtrar por categorías relevantes
- Mejor tras el ajuste de umbral, pero únicamente en el caso del dataset completo, ya que al ajustar el umbral de las características más relevantes, el resultado fue el umbral por defecto.

Concluimos por tanto que este modelo presenta un rendimiento aceptable sobre el dataset representado en variables mayoritariamente binarias o booleanas, y desbalanceado para la clase objetivo. En todos los casos, la detección de la clase minoritaria se ve penalizada, lo que sugiere que 'auto_class_weigths' no resulta un método suficientemente efectivo para paliar el desbalanceo, a pesar de que presente mejores resultados que SMOTE. 

Por otro lado, el modelo ha sido entrenado sin la influencia de dos variables textuales: `statement`y `subject`, cuyo posible efecto en el aprendizaje no se encuentra representado debido a las dificultades del modelo para interpretarlas. Además, en este modo, CatBoost no presenta características para procesar estas variables obteniendo contexto lingüístico ni relaciones complejas por lo que surge la necesidad de encontrar otra estrategia que facilite a CatBoost el aprendizaje a través del lenguaje. 

